In [145]:

import os
import time

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
from torch.utils.data import DataLoader
from torchvision import transforms, datasets
from tqdm.notebook import tqdm_notebook as tqdm

# matplotlib.style.use("ggplot")

device = "cuda" if torch.cuda.is_available() else "cpu"

In [146]:
train_transform = transforms.Compose(
    [
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(),
        transforms.RandomVerticalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225]
        ),
    ]
)
val_transform = transforms.Compose(
    [
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225]
        ),
    ]
)

In [148]:
data_root = "../data/isdog"
train_dataset = datasets.ImageFolder(
    root=data_root + "/train",
    transform=train_transform
)

train_dataloader = DataLoader(
    train_dataset, batch_size=32, shuffle=True
)

val_dataset = datasets.ImageFolder(
    root=data_root + "/test",
    transform=val_transform
)

val_dataloader = DataLoader(
    val_dataset, batch_size=32, shuffle=False
)

In [149]:
def resnet50(pretrained=True, require_grad=False):
    model = models.resnet50(pretrained=pretrained, progress=True)
    if not require_grad:
        for param in model.parameters():
            param.requires_grad = False
    else:
        for param in model.parameters():
            param.requires_grad = True
    model.fc = nn.Linear(2048, 2)
    return model

In [150]:
class LRScheduler:
    def __init__(self, opti, patience=5, min_lr=1e-6, factor=0.5):
        self.opti = opti
        self.patience = patience
        self.min_lr = min_lr
        self.factor = factor
        self.lr_scheduler = optim.lr_scheduler.ReduceLROnPlateau(
            self.opti, mode="min", patience=self.patience, factor=self.factor, verbose=True
        )

    def __call__(self, val_loss):
        self.lr_scheduler.step(val_loss)

In [151]:
class EarlyStopping():
    def __init__(self, patience=5, verbose=True, delta=0, path="ckpt/isdog_resnet50.pt"):
        self.patience = patience
        self.counter = 0
        self.verbose = verbose
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.delta = delta
        self.path = path

    def __call__(self, val_loss, model):
        score = -val_loss
        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        elif score < self.best_score + self.delta:
            self.counter += 1
            print(f"EarlyStopping counter: {self.counter} out of {self.patience}")
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        if self.verbose:
            print(f"Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}). Saving model ...")

        # create folder if it doesn't exist
        if not os.path.isdir("ckpt"):
            os.mkdir("ckpt")
            
        torch.save(model.state_dict(), self.path)
        self.val_loss_min = val_loss

In [128]:
# parser = argparse.ArgumentParser()
# parser.add_argument("--lr", desc="lr_scheduler", action="store_true")
# parser.add_argument("--es", desc="early_stopping", action="store_true")
# 
# args = vars(parser.parse_args())

In [152]:
print(f"Computation device: {device}")
model = models.resnet50(pretrained=True)
model.to(device)
total_params = sum(p.numel() for p in model.parameters())
print(f"Total parameters: {total_params:}")
total_trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Total trainable parameters: {total_trainable_params:}")

Computation device: cuda


C:\Program Files\Python311\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Program Files\Python311\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Total parameters: 25557032
Total trainable parameters: 25557032


In [153]:
lr = 0.001
epochs = 100
optimizer = optim.Adam(model.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss()

In [159]:
loss_plot_name = "loss"
acc_plot_name = "accuracy"
model_name = "isdog_resnet50"

use_scheduler = bool(input("Use lr scheduler? (y/n)").lower() == "y")
use_early_stopping = bool(input("Use early stopping? (y/n)").lower() == "y")

if use_scheduler:
    lr_scheduler = LRScheduler(optimizer)
    print("Using lr scheduler")
    loss_plot_name += "lrs_loss"
    acc_plot_name += "lrs_acc"
    model_name += "lrs"

if use_early_stopping:
    early_stopping = EarlyStopping()
    print("Using early stopping")
    loss_plot_name += "es_loss"
    acc_plot_name += "es_acc"
    model_name += "es"

In [155]:
def training(model, train_dataloader, train_dataset, optimizer, criterion):
    print("Training")
    model.train()
    r_loss = 0.0
    r_correct = 0

    counter = 0
    total = 0

    prog_bar = tqdm(enumerate(train_dataloader), total=int(len(train_dataset) / train_dataloader.batch_size))

    for i, data in prog_bar:
        counter += 1
        data, target = data[0].to(device), data[1].to(device)
        total += target.size(0)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        r_loss += loss.item()
        _, preds = torch.max(output, 1)
        r_correct += (preds == target.data).sum().item()
        loss.backward()
        optimizer.step()

    train_loss = r_loss / counter
    train_accuracy = 100. * r_correct / total
    return train_loss, train_accuracy

In [156]:
def validate(model, test_dataloader, val_dataset, criterion):
    print("Validating")

    model.eval()

    r_loss = 0.0
    r_correct = 0

    counter = 0
    total = 0

    prog_bar = tqdm(enumerate(test_dataloader), total=int(len(val_dataset) / test_dataloader.batch_size))

    with torch.no_grad():
        for i, data in prog_bar:
            counter += 1
            data, target = data[0].to(device), data[1].to(device)
            total += target.size(0)
            outputs = model(data)
            loss = criterion(outputs, target)

            r_loss += loss.item()
            _, preds = torch.max(outputs, 1)
            r_correct += (preds == target.data).sum().item()

    val_loss = r_loss / counter
    val_acc = 100. * r_correct / total
    return val_loss, val_acc

In [134]:
train_loss, train_acc = [], []
val_loss, val_acc = [], []

start_at = time.time()

for epoch in range(epochs):
    print(f"Epoch {epoch + 1}/{epochs}")
    train_epoch_loss, train_epoch_acc = training(model, train_dataloader, train_dataset, optimizer, criterion)
    val_epoch_loss, val_epoch_acc = validate(model, val_dataloader, val_dataset, criterion)

    train_loss.append(train_epoch_loss)
    train_acc.append(train_epoch_acc)

    if use_scheduler:
        lr_scheduler(val_epoch_loss)
    if use_early_stopping:
        early_stopping(val_epoch_loss, model)
        if early_stopping.early_stop:
            print("Early stopping")
            break
    print("Train loss: {:.4f}, Train accuracy: {:.2f}%".format(train_epoch_loss, train_epoch_acc))
    print("Validation loss: {:.4f}, Validation accuracy: {:.2f}%".format(val_epoch_loss, val_epoch_acc))
    end = time.time()
    print(f"Time elapsed: {end - start_at:.3f}s")

Epoch 1/100
Training


  0%|          | 0/15 [00:00<?, ?it/s]

Validating


  0%|          | 0/15 [00:00<?, ?it/s]

Train loss: 2.1243, Train accuracy: 59.44%
Validation loss: 89.4616, Validation accuracy: 52.60%
Time elapsed: 8.215s
Epoch 2/100
Training


...

Train loss: 0.0026, Train accuracy: 100.00%
Validation loss: 0.6383, Validation accuracy: 84.40%
Time elapsed: 798.367s
Epoch 100/100
Training


  0%|          | 0/15 [00:00<?, ?it/s]

Validating


  0%|          | 0/15 [00:00<?, ?it/s]

Train loss: 0.0037, Train accuracy: 100.00%
Validation loss: 0.6535, Validation accuracy: 83.40%
Time elapsed: 807.270s


In [161]:
if not os.path.isdir("plots"):
    os.mkdir("plots")

print("Saving loss and accuracy plots")
plt.figure(figsize=(10, 7))
plt.plot(train_loss, color="orange", label="train loss")
plt.plot(val_loss, color="red", label="validation loss")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend()
plt.savefig(f"plots/{loss_plot_name}.png")
plt.show()

plt.figure(figsize=(10, 7))
plt.plot(train_acc, color="orange", label="train accuracy")
plt.plot(val_acc, color="red", label="validation accuracy")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend()
plt.savefig(f"plots/{acc_plot_name}.png")
plt.show()

print("Saving model")
torch.save(model.state_dict(), f"ckpt/{model_name}.pt")

Saving loss and accuracy plots
Saving model


In [158]:
print("COMPLETED")

COMPLETED
